In [1]:
import dadrah.analysis.root_plotting_util as rpu
import dadrah.util.run_paths as runpa
import dadrah.util.string_constants as stco
import pofah.jet_sample as js

import pathlib
import argparse
from typing import List

import numpy as np
import ROOT as rt
import root_numpy as rtnp
import uuid
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import root_numpy as rtnp
import mplhep as hep
plt.style.use(hep.style.ROOT)
from typing import List

Welcome to JupyROOT 6.18/04


## run config

In [2]:
ae_run_n = 113
qr_run_n, qr_model_str = 4, 'poly'
xsec_train = 0 # signal cross section used when training the QR
xsecs_sig   = [0,20,40,60,80,100] # signal cross sections
mX = 3.5
sample_ids = ['qcdSigAllTestReco', 'GtoWW35naReco']
sample_id_qcd = sample_ids[0]
sample_id_sig = sample_ids[1]
quantiles = [0.1, 0.3, 0.5, 0.7, 0.9, 0.99]
#quantiles = [0.3, 0.5, 0.7, 0.9, 0.99]

# choose which template quantile to use (0 = 0.9-1.0, 1 = 0.7-0.9)
template_q_idx = 0
# Set binning
dijet_bin = True

In [3]:
# Rescaling the inputs to lumi
xsec_qcd       = 8730000.0 # Crossection in fb             
qcd_gen_events = 134366091.0+199435365.0+90490645.0+134264102.0 #(all generated QCD)
xsec_sig_default = 10. #In units of fb (10 fb == 0.01 pb) 
sig_gen_events = 982038.0 #(all generated signal)
lumi           = qcd_gen_events/xsec_qcd # assuming 64/fb since qcd_gen_events/8730000 ~ 64

scale_qcd = xsec_qcd*lumi/qcd_gen_events
scale_sig = 0.8*xsec_sig_default*lumi/sig_gen_events #20% already used for quantile regression, must be removed
print('Usign luminosity {}'.format(lumi))
print('Scale {} QCD events with {}'.format(qcd_gen_events,scale_qcd))
print('Scale {} signal events with {} for 10fb default sig xsec'.format(sig_gen_events,scale_sig))

Usign luminosity 63.98123745704467
Scale 558556203.0 QCD events with 1.0
Scale 982038.0 signal events with 0.0005212119079468996 for 10fb default sig xsec


# read in data

### train xsec = 0 !

In [4]:
paths = runpa.RunPaths(in_data_dir=stco.dir_path_dict['base_dir_qr_selections'], in_data_names=stco.file_name_path_dict, out_data_dir=stco.dir_path_dict['base_dir_qr_analysis'])
path_ext_dict = {'vae_run': str(ae_run_n), 'qr_run': str(qr_run_n), 'sig': sample_id_sig, 'xsec': str(int(xsec_train)), 'loss': 'rk5_05'}
paths.extend_in_path_data(path_ext_dict)
paths.extend_out_path_data({**path_ext_dict, 'mjj_spectra': None, 'orthogonal': None})

samples = {}

for sample_id in sample_ids:
    samples[sample_id] = js.JetSample.from_input_file(sample_id, paths.in_file_path(sample_id))

### divide into orthogonal quantiles

In [5]:
def divide_sample_into_orthogonal_quantiles(sample:js.JetSample, quantiles:List) -> List[js.JetSample]:

    quantiles_inv = [round((1.-q),2) for q in quantiles] # invert quantiles to [0.9, 0.7, 0.5, 0.3, 0.1, 0.01]

    samples_ortho = []

    # process bottom quantile (didn't make first cut, e.g. sel 0.9 == 0)
    q_key = 'sel_q{:02}'.format(int(quantiles_inv[0]*100))
    samples_ortho.append(sample.filter(~sample[q_key]))

    # process all quantiles except for last (tightest)
    for q_i, q_ii in zip(quantiles_inv[:-1], quantiles_inv[1:]):

        q_i_key, q_ii_key = 'sel_q{:02}'.format(int(q_i*100)), 'sel_q{:02}'.format(int(q_ii*100))

        sample_q_next = sample.filter(sample[q_i_key] & ~sample[q_ii_key])
        samples_ortho.append(sample_q_next)

    # process tightest quantile
    q_key = 'sel_q{:02}'.format(int(quantiles_inv[-1]*100))
    samples_ortho.append(sample.filter(sample[q_key]))

    return samples_ortho

In [6]:
# qcd
samples_ortho_quantiles_qcd = divide_sample_into_orthogonal_quantiles(samples[sample_id_qcd], quantiles)
mjj_vals_qcd = [sample_ortho['mJJ'] for sample_ortho in samples_ortho_quantiles_qcd]
# signal
samples_ortho_quantiles_sig = divide_sample_into_orthogonal_quantiles(samples[sample_id_sig], quantiles)
mjj_vals_sig = [sample_ortho['mJJ'] for sample_ortho in samples_ortho_quantiles_sig]

In [7]:
num_samples_qcd = sum(map(len,mjj_vals_qcd))
print(num_samples_qcd)
num_samples_sig = sum(map(len,mjj_vals_sig))
print(num_samples_sig)

7671759
531825


In [8]:
quantiles_inv = [round((1.-q),2) for q in [0]+quantiles] # invert quantiles to [1.0 0.9, 0.7, 0.5, 0.3, 0.1, 0.01]
q_str_leg = [str(q) for q in quantiles_inv+[0.]]

In [9]:
q_str_leg

['1.0', '0.9', '0.7', '0.5', '0.3', '0.1', '0.01', '0.0']

In [10]:
# Dijet variable-binwidth binning (depends on dijet resolution)
if dijet_bin:
    bin_edges = np.array([1200, 1255, 1320, 1387, 1457, 1529,
                          1604, 1681, 1761, 1844, 1930, 2019, 
                          2111, 2206, 2305, 2406, 2512, 2620, 
                          2733, 2849, 2969, 3093, 3221, 3353,
                          3490, 3632, 3778, 3928, 4084, 4245, 
                          4411, 4583, 4760, 4943, 5132, 5327]).astype('float')
                          #5574, 5737, 5951, 6173, 6402, 6638, 6882]).astype('float')
# Constant binwidth
else:
    bin_edges = np.array(np.arange(1200.,6800., 50.).tolist()).astype('float') #100 GeV binning. Stop at 5600! Fit fails if going to 6800


n_bins = len(bin_edges)-1
max_bin = bin_edges[-1]
min_bin = bin_edges[0]

### color palettes

In [11]:
cmap_qcd = cm.get_cmap('winter') # 'summer'
cmap_qcd = [mpl.colors.rgb2hex(c) for c in cmap_qcd(np.linspace(0.0, 1.0, len(mjj_vals_qcd)+1))][1:]
cmap_sig = cm.get_cmap('winter')
cmap_sig = [mpl.colors.rgb2hex(c) for c in cmap_sig(np.linspace(0.0, 1.0, len(mjj_vals_sig)+1))][1:]
bump_col = '#e25650' # colors for signal bump

## fill events into histograms

In [12]:
n_bins = len(bin_edges)-1 #50
hist_sz = 700
marker_sty = 8
marker_sz = 0.85

In [13]:
datas_qcd = {}
datas_sig = {}

# qcd template / bottom histogram
props={'LineColor': rt.kBlack, "YTitle": 'num events', "XTitle": "M_{jj} [GeV]", "Title":"", "Minimum": 0.2, "Maximum": num_samples_qcd*1.1}
datas_qcd[quantiles_inv[template_q_idx]] = rpu.create_hist(mjj_vals_qcd[template_q_idx],'',n_bins, bin_edges.astype('float'), props=props)

# qcd other ortho quantile hists
for mjj, q, c in zip(mjj_vals_qcd[template_q_idx+1:], quantiles_inv[template_q_idx+1:], cmap_qcd[template_q_idx:]):
    props = {"LineColor": rt.TColor.GetColor(c), "MarkerColor": rt.TColor.GetColor(c), "LineWidth": 1, "MarkerStyle": marker_sty, "MarkerSize": marker_sz}
    datas_qcd[q] = rpu.create_hist(mjj, '', n_bins, bin_edges.astype('float'), props=props)

# signal orthogonal quantiles
for mjj, q, in zip(mjj_vals_sig[template_q_idx:], quantiles_inv[template_q_idx:]):
    props = {"LineColor": rt.TColor.GetColor(bump_col), "LineColorAlpha": ((rt.TColor.GetColor(bump_col)), q), "FillColorAlpha": ((rt.TColor.GetColor(bump_col)), q*0.8), "LineWidth": 1,}
    datas_sig[q] = rpu.create_hist(mjj, '', n_bins, bin_edges.astype('float'), props=props)

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure al

## make histograms with injected signal pseudo data

In [14]:
# this is all for training signal xsec = 0

histos_data_inj = {}
histos_sig_inj = {}

# for each quantile
for q in quantiles_inv:
    
    q_str = 'q'+str(int(q*100))
    histos_data_inj_quantile = {}
    histos_sig_inj_quantile = {}
    
    # for each signal cross section (each signal injection value)
    for xsec in xsecs_sig:
        
        sig_inj = int(xsec/10)
        print('xsec {}, sig_inj {}'.format(xsec, sig_inj))
        hist_data_xsec = datas_qcd[q].Clone('data_hist_acc_inj{}_{}'.format(sig_inj,q_str)) # get qcd events for quantile q
        hist_sig_xsec = datas_sig[q].Clone('signal_inj{}_{}'.format(sig_inj,q_str)) # get signal events for quantile q
        tmpSig = hist_sig_xsec.Clone('tmp_signal_inj{}_{}'.format(sig_inj,q_str))
        
        # Scale signal by the crossection injected (default is 10 fb)
        print("Signal before scaling = {}".format(tmpSig.Integral()))
        tmpSig.Scale(scale_sig*sig_inj)
        print("Signal after scaling = {}".format(tmpSig.Integral()))
        
        hist_data_xsec.Sumw2() # Sumw2: tell the histogram class to also store errors.
        tmpSig.Sumw2()
        hist_sig_xsec.Sumw2()
        
        rdm = rt.TRandom3(0)
        for i in range (0,tmpSig.GetNbinsX()):
            n = tmpSig.GetBinContent(i+1)  
            n_rdm = rdm.Poisson(n)
            hist_sig_xsec.SetBinContent(i+1,n_rdm)
            hist_sig_xsec.SetBinError(i+1,n_rdm**0.5)
        print("Signal after pseudodata = {}".format(hist_sig_xsec.Integral()))
    
        # add pseudo signal events to qcd events. 
        if sig_inj != 0:
            hist_data_xsec.Add(hist_sig_xsec)

        hist_data_xsec.SetBinErrorOption(rt.TH1.kPoisson)
        
        # save mixed data histo and signal histo in quantile dict
        histos_data_inj_quantile[sig_inj] = hist_data_xsec
        histos_sig_inj_quantile[sig_inj] = hist_sig_xsec
        
    # save full xsec set for each quantile
    histos_data_inj[q] = histos_data_inj_quantile
    histos_sig_inj[q] = histos_sig_inj_quantile


xsec 0, sig_inj 0
Signal before scaling = 21954.0
Signal after scaling = 0.0
Signal after pseudodata = 0.0
xsec 20, sig_inj 2
Signal before scaling = 21954.0
Signal after scaling = 22.88537245413246
Signal after pseudodata = 28.0
xsec 40, sig_inj 4
Signal before scaling = 21954.0
Signal after scaling = 45.77074490826492
Signal after pseudodata = 42.0
xsec 60, sig_inj 6
Signal before scaling = 21954.0
Signal after scaling = 68.65611736239737
Signal after pseudodata = 78.0
xsec 80, sig_inj 8
Signal before scaling = 21954.0
Signal after scaling = 91.54148981652985
Signal after pseudodata = 91.0
xsec 100, sig_inj 10
Signal before scaling = 21954.0
Signal after scaling = 114.42686227066237
Signal after pseudodata = 103.0
xsec 0, sig_inj 0
Signal before scaling = 43293.0
Signal after scaling = 0.0
Signal after pseudodata = 0.0
xsec 20, sig_inj 2
Signal before scaling = 43293.0
Signal after scaling = 45.12965426149025
Signal after pseudodata = 50.0
xsec 40, sig_inj 4
Signal before scaling = 4

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure al

# plot ratio hist for each quantile with increasing xsec added

In [15]:
template_q_idx = 0
template_q = quantiles_inv[template_q_idx]
sig_injs = [int(xsec/10) for xsec in xsecs_sig]
sig_inj = sig_injs[-1]

In [16]:
import cmsstyle #mplhep for Python2 cheatsheet

# for each signal injection value
for sig_inj in sig_injs:

    c_out, pad1, pad2 = rpu.create_canvas_pads(pad_props={'LeftMargin': 0.13})
    
    # legend
    leg = rpu.create_object("TLegend", 0.45, 0.5, 0.87, 0.78)
    leg.SetNColumns(2)
    leg_h = rpu.create_object("TLegend", 0.35, 0.8, 0.95, 0.85) # second legend for header because left align not available
    rpu.set_style(leg,props={'TextSize': 0.045})
    rpu.set_style(leg_h,props={'TextSize': 0.045})

    ### plot data histograms

    pad1.cd()

    # draw template bottom quantile data and signal histogram
    ht = histos_data_inj[template_q][sig_inj] 
    ht.Draw('HIST')
    leg.AddEntry(ht, 'q = '+q_str_leg[template_q_idx+1]+'-'+q_str_leg[template_q_idx], 'lep')
    hs = histos_sig_inj[template_q][sig_inj]
    hs.Draw('hist same')
    leg.AddEntry(hs, 'q =  '+q_str_leg[template_q_idx+1]+'-'+q_str_leg[template_q_idx], 'l')


    # draw all other quantile histograms
    for q, q_str_i, q_str_ii in zip(quantiles_inv[template_q_idx+1:], q_str_leg[template_q_idx+2:], q_str_leg[template_q_idx+1:]):
        # data
        h = histos_data_inj[q][sig_inj]
        h.Draw('same')
        leg.AddEntry(h, 'q = {} - {}'.format(q_str_i,q_str_ii), 'lep')
        # signal
        hs = histos_sig_inj[q][sig_inj]
        hs.Draw('hist same')
        leg.AddEntry(hs, 'q = {} - {}'.format(q_str_i,q_str_ii), 'l')

    leg_h.SetHeader(r'Data: QCD MC + G_{RS}(%.1fTeV, #sigma=%i fb)'%(mX,sig_inj*10))
    leg.Draw()
    leg_h.Draw()

    # create ratio histograms
    hr = []
    for q, c in zip(quantiles_inv[1:], cmap_qcd):
        h = histos_data_inj[q][sig_inj]
        props={"LineColor": rt.TColor.GetColor(c), "MarkerColor": rt.TColor.GetColor(c), "Title": '', "LineWidth": 1, "MarkerStyle": marker_sty}
        r = rpu.create_ratio_hist(h,histos_data_inj[template_q][sig_inj],props=props)
        hr.append(r)
    rpu.set_style(hr[0], props={ "XTitle": 'M_{jj} [GeV]', "YTitle": "ratio SIG / BG"})

    pad2.cd()
    hr[0].Draw('pe')
    props={'YTitle':'#frac{#epsilon#timesq}{q_{'+q_str_leg[template_q_idx+1]+'-'+q_str_leg[template_q_idx]+'}}', 'XTitle':'M_{jj} (GeV)', 'Maximum':2.2, 'Minimum':0., 'TitleOffset':0.2}
    rpu.set_style(hr[0],props=props)
    rpu.set_style(hr[0].GetYaxis(),props={'TitleSize':0.11, 'LabelSize':0.12, 'Ndivisions':304, 'TitleOffset':0.4 })
    rpu.set_style(hr[0].GetXaxis(),props={'TitleSize':0.11, 'LabelSize':0.12, 'Ndivisions':304, 'TitleOffset':0.95, 'Ndivisions':909, 'TickSize':0.07 })
    hr[0].GetYaxis().CenterTitle()

    for r in hr[1:]:
        r.Draw('pe same')

    eff_line = rpu.create_ratio_line(hr[0])
    eff_line.Draw('same')

    c_out.Draw()
    c_out.SaveAs('fig/mjj_ratio_inj_sig_xsec_{}.pdf'.format(sig_inj*10))

ModuleNotFoundError: No module named 'cmsstyle'

In [ ]:
for inj in sig_injs:
    
    sum_sig_inj = sum(histos_sig_inj[q][inj].Integral() for q in quantiles_inv)
    sum_data = sum(histos_data_inj[q][inj].Integral() for q in quantiles_inv)

    for q in quantiles_inv:
        print('sig inj {}, q {}: {} signal events'.format(inj, q, histos_sig_inj[q][inj].Integral()))

    print('total injected sig events for xsec {}: {} ({:.4f}% of {} data events)'.format(inj*10, sum_sig_inj, (sum_sig_inj/sum_data)*100, sum_data))


## write out histogram files

In [ ]:
quantiles_inv

In [ ]:
# Make one file for shortcut-data

tag = 'PDinj_NA3p5'
sig_injs = [0,2,4,6,8,10]

outfile = rt.TFile("histograms_injSig_{}.root".format(tag), 'RECREATE' )

for si in sig_injs:
    xs = 10*si    
    for q in quantiles_inv:
        quantile = int(q*100)
        print("For xsec {}, q {}".format(xs,quantile))
        histos_data_inj[q][si].Write('data_acc_{}fb_q{}p'.format(xs,quantile))
        histos_sig_inj[q][si].Write('signal_acc_{}fb_q{}p'.format(xs,quantile))

outfile.Close() 